In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 2.7.15+ (default, Nov 27 2018, 23:36:35) 
[GCC 7.3.0]
NLTK: 3.2.5
Scikit-learn: 0.20.3
Pandas: 0.24.2
Numpy: 1.14.1


In [2]:
# Carregando dados
import pandas as pd
import numpy as np

df = pd.read_csv('data/dataset_tratado.txt', header=None, encoding='utf-8', sep='\t')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null bool
1    793 non-null object
dtypes: bool(1), object(1)
memory usage: 7.0+ KB
None
       0                                                  1
0  False  mand segur inexecu contrat sançã afast reexam ...
1  False  respons civil acident aér decret lei n dol eve...
2  False  processual civil intim art cpc irregular compr...
3  False  process civil honorári execu legitim part supe...
4  False  administr servidor públic artig lei n artig le...


In [9]:
# verificação das classes dos acórdãos

classes = df[0]
print(classes.value_counts())

False    525
True     268
Name: 0, dtype: int64


In [10]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes em binário
encoder = LabelEncoder()
binary = encoder.fit_transform(classes)

print(binary[:10])

[0 0 0 0 0 0 1 1 0 0]


In [11]:
# criação da lista de ementas
ementas = df[1]
print(ementas[:10])

0    mand segur inexecu contrat sançã afast reexam ...
1    respons civil acident aér decret lei n dol eve...
2    processual civil intim art cpc irregular compr...
3    process civil honorári execu legitim part supe...
4    administr servidor públic artig lei n artig le...
5    impost sobr servic iss arrend mercantil incide...
6    petiçã receb hab corpus processual penal art c...
7    penal process penal recurs especial art incis ...
8    tribut sujeit lançament homolog praz prescrici...
9    transport intermunicipal licit açã anulatór in...
Name: 1, dtype: object


In [12]:
# The following script uses the bag of words model to convert text documents into 
# corresponding numerical features:

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = vectorizer.fit_transform(ementas).toarray()

print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
# directly convert text documents into TFIDF feature values 
# (without first converting documents to bag of words features)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(ementas).toarray()

print(X)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.05510598]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [40]:
# Like any other supervised machine learning problem, we need to divide data into 
# 20% test set and 80% training set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, binary, test_size=0.2, random_state=0)

In [41]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print('accuracy %s' % (accuracy_score(y_pred, y_test)*100))
print(classification_report(y_test, y_pred,target_names=['deferido', 'indeferido']))

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [32]:
# Training the model and predicting pentiment with Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)

print('accuracy %s' % (accuracy_score(y_pred, y_test)*100))
print(classification_report(y_test, y_pred,target_names=['deferido', 'indeferido']))

accuracy 88.0503144654088
              precision    recall  f1-score   support

    deferido       0.86      0.96      0.91       100
  indeferido       0.92      0.75      0.82        59

   micro avg       0.88      0.88      0.88       159
   macro avg       0.89      0.85      0.87       159
weighted avg       0.88      0.88      0.88       159



In [30]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

%%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

NameError: name 'Pipeline' is not defined

In [26]:
# To evaluate the performance of a classification model we use metrics the confusion matrix, 
# F1 measure, and the accuracy.

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[147   3]
 [ 38  50]]
              precision    recall  f1-score   support

           0       0.79      0.98      0.88       150
           1       0.94      0.57      0.71        88

   micro avg       0.83      0.83      0.83       238
   macro avg       0.87      0.77      0.79       238
weighted avg       0.85      0.83      0.82       238

0.8277310924369747


In [ ]:
# to predict new document use

model.predict( [" Enter your sentence here "] )